# Google Scholar - Coventry University potential reasearch supervisors for postgraduate research

In [1]:
#Import the required libraries
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from time import sleep
from random import randint
import unicodedata
import re

## Crawling first page 

Using the the seed URL to crawl the first page 
1. Get part of the weblink to crawl individual professor/personnel page
2. Combine part of the URL to the main google scholar link
3. From this page get the name, research category
4. Combine the person informations into dataframe and save it as a CSV file to easy append all pages data
5. Get the link to the next page, google scholar have this not normalize using character replacement and
    unicode normalization to transform into normal string 

In [6]:
stflinks = []
stfname = []
name = []
coven = pd.DataFrame(columns= ['name', 'link','research_field'])
page1 = requests.get("https://scholar.google.co.uk/citations?view_op=view_org&hl=en&org=9117984065169182779")
soup1 = BeautifulSoup(page1.text, 'html.parser')
#sub link to a staff page

for main in soup1.find_all('div', class_='gsc_1usr'):
    try:
        slink = main.h3.a['href']
    except Exception as e:
        slink = 'None' 
    stflinks.append(slink)
    
#staff page extract details(name and research area)   
for stfpage in stflinks:
    pagestf= requests.get("https://scholar.google.co.uk" + str(stfpage))
    soup2 = BeautifulSoup(pagestf.text, 'html.parser')                      
    try:
        sname = soup2.find('div', id ='gsc_prf_in').text
    except Exception as e:
        sname = 'None'  
        
#all area research in one row         
    stfarea = []
    for main1 in soup2.find_all(class_='gsc_prf_inta gs_ibl'):
        try:
            stfar = main1.text
        except Exception as e:
            stfar = 'None'
        #stfarea.append('\n')    
        stfarea.append(stfar)
        
       
    links = ("https://scholar.google.co.uk" + str(stfpage))
    string = ", "
    string = string.join(stfarea)
#Dataframe     
    a = {'names': sname, 'link' : links, 'research_field': string}
    cov = pd.DataFrame.from_dict(a, orient='index')
    coventry = cov.transpose()    
    coven = coven.append(coventry, ignore_index= 'True')
    coven.to_csv('coventry1.csv')


for main3 in soup1.find_all('div', class_= 'gsc_pgn'):
    for main3 in soup1.find_all(class_= 'gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx'):
        try:
            nextlink = main3['onclick']            
        except Exception as e:
            nextlink = 'None' 
        start_pos = nextlink.find('=')
        temp= nextlink[start_pos + 2:len(nextlink) -1]
        
def normalize_string(text):
    
    text = text.replace("\\x3d", "=")    
    text = text.replace("\\x26", "&")    
    text = text.strip()

    text = text.rstrip('.')
    text = unicodedata.normalize(u'NFKD', str(text)).encode('ascii', 'ignore').decode('utf8')
    
    return text 
normalize_string(temp)

link = 'https://scholar.google.co.uk' + normalize_string(temp)
print(link)

https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=9117984065169182779&after_author=zMpCAD_n__8J&astart=10


In [7]:
coven.head()

,name,link,research_field
0,Timothy Mason,https://scholar.google.co.uk/citations?hl=en&u...,"sonochemistry, ultrasound, chemistry, environm..."
1,Gurnam Singh,https://scholar.google.co.uk/citations?hl=en&u...,"social work, race and racism, critical pedagog..."
2,WD Li,https://scholar.google.co.uk/citations?hl=en&u...,
3,Dr. Mohammad M Ali,https://scholar.google.co.uk/citations?hl=en&u...,"Forecast Information Sharing, ARIMA Modelling,..."
4,Petra Wark,https://scholar.google.co.uk/citations?hl=en&u...,"m/eHealth, epidemiology, primary prevention, d..."


From the link above 
1. Using a while loop to crawl through the rest of the pages
2. using a sleep time with random generated time between 1 and 5 seconds to pause crawling for the rest of the website in respect to other people using it, not to over whelm the website and respectiful behaviour
3. There 68 pages on this websitw with 10 people per page, each page crawled data is appended to the CSV file  
4. Printed every URL just a check on the code progress make sure its not in a infinity loop 

In [8]:
i= 1
while i: 
    if nextlink != 'None':
        stflinks = []
        stfname = []
        name = []
        coven = pd.DataFrame(columns= ['name', 'link','research_field'])
        page1 = requests.get(link)
        soup1 = BeautifulSoup(page1.text, 'html.parser')
        #sub link to a staff page

        for main in soup1.find_all('div', class_='gsc_1usr'):
            try:
                slink = main.h3.a['href']
            except Exception as e:
                slink = 'None' 
            stflinks.append(slink)

        #staff page extract details(name and research area)   
        for stfpage in stflinks:
            pagestf= requests.get("https://scholar.google.co.uk" + str(stfpage))
            soup2 = BeautifulSoup(pagestf.text, 'html.parser')                      
            try:
                sname = soup2.find('div', id ='gsc_prf_in').text
            except Exception as e:
                sname = 'None'  

        #all area research in one row         
            stfarea = []
            for main1 in soup2.find_all(class_='gsc_prf_inta gs_ibl'):
                try:
                    stfar = main1.text
                except Exception as e:
                    stfar = 'None'
                #stfarea.append('\n')
                stfarea.append(stfar)


            links = ("https://scholar.google.co.uk" + str(stfpage))
            string = ", "
            string = string.join(stfarea)
        #Dataframe     
            a = {'names': sname, 'link' : links, 'research_field': string}
            cov = pd.DataFrame.from_dict(a, orient='index')
            coventry = cov.transpose()    
            coven = coven.append(coventry, ignore_index= 'True')
        coven.to_csv('coventry1.csv', mode='a', header=False)

        #Nextpage link
        for main3 in soup1.find_all('div', class_= 'gsc_pgn'):
            for main3 in soup1.find_all(class_= 'gs_btnPR gs_in_ib gs_btn_half gs_btn_lsb gs_btn_srt gsc_pgn_pnx'):
                try:
                    nextlink = main3['onclick']            
                except Exception as e:
                    nextlink = 'None' 
                start_pos = nextlink.find('=')
                temp= nextlink[start_pos + 2:len(nextlink) -1]

        def normalize_string(text):

            text = text.replace("\\x3d", "=")    
            text = text.replace("\\x26", "&")    
            text = text.strip()

            text = text.rstrip('.')
            text = unicodedata.normalize(u'NFKD', str(text)).encode('ascii', 'ignore').decode('utf8')

            return text 
        normalize_string(temp)

        link = 'https://scholar.google.co.uk' + normalize_string(temp)
        print(link)
    else:
        break
    
    sleep(randint(1,5))        
    i += 1   

https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=9117984065169182779&after_author=9Os8AKnu__8J&astart=20
https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=9117984065169182779&after_author=f0osAKXy__8J&astart=30
https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=9117984065169182779&after_author=CVhmAGT0__8J&astart=40
https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=9117984065169182779&after_author=GY4kAOb2__8J&astart=50
https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=9117984065169182779&after_author=PAQYAHT4__8J&astart=60
https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=9117984065169182779&after_author=M-peAKn5__8J&astart=70
https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=9117984065169182779&after_author=qtSrAFf6__8J&astart=80
https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=91179840

https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=9117984065169182779&after_author=LAh4APz___8J&astart=650
https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=9117984065169182779&after_author=5ixoAP3___8J&astart=660
https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=9117984065169182779&after_author=H03SAP3___8J&astart=670
https://scholar.google.co.uk/citations?view_op=view_org&hl=en&oe=ASCII&org=9117984065169182779&after_author=C2oHAf3___8J&astart=680
https://scholar.google.co.ukon


# Get Coventry University Faculty data

The faulty page do not have research interest so this has to be crawled differently 
Have to crawl different faculty personnel separa

### Faculty of Arts and Humanities¶

In [9]:
faculty= []
subfacul= []
pnames = []
nextpag = []

page4 = requests.get("https://pureportal.coventry.ac.uk/en/organisations/faculty-of-arts-humanities/persons/")
soup4 = BeautifulSoup(page4.text, 'html.parser')


for main4 in soup4.find_all('div', class_='organisation-details'):
    try:
        facul = main4.h1.text
    except Exception as e:
        facul = 'None'
    faculty.append(facul)    
    
for main5 in soup4.find_all('h3', class_='title'):
    try:
        pers = main5.text
    except Exception as e:
        pers = 'None'
    pnames.append(pers)    
        
for main6 in soup4.find_all('ul',class_='relations organisations'):
    try:
        perfacul = main6.span.text
    except Exception as e:
        perfacul = 'None'
    subfacul.append(perfacul)

#Nextpage link
for main7 in soup4.find_all('li', class_='next'):
    try:
        nextpage = main7.a['href']
    except Exception as e:  
        nextpage = 'None'
    #nextpag.append(nextpage)     
    
b ={'mainfaculty':faculty, 'pernames':pnames,'subfaculty':subfacul}
cov_facul = pd.DataFrame.from_dict(b, orient='index')
cov_facult = cov_facul.transpose()
cov_facult['faculty'] =cov_facult['mainfaculty'].iloc[0]
cov_facult.to_csv('cov_faculty_art.csv')
    
nextpagelink = ("https://pureportal.coventry.ac.uk" + str(nextpage))       
print(nextpagelink)


https://pureportal.coventry.ac.uk/en/organisations/faculty-of-arts-humanities/persons/?page=1


In [10]:
j = 1
while j < 6:
    faculty= []
    subfacul= []
    pnames = []
    nextpag = []

    page4 = requests.get(nextpagelink)
    soup4 = BeautifulSoup(page4.text, 'html.parser')


    for main4 in soup4.find_all('div', class_='organisation-details'):
        try:
            facul = main4.h1.text
        except Exception as e:
            facul = 'None'
        faculty.append(facul)    

    for main5 in soup4.find_all('h3', class_='title'):
        try:
            pers = main5.text
        except Exception as e:
            pers = 'None'
        pnames.append(pers)    

    for main6 in soup4.find_all('ul',class_='relations organisations'):
        try:
            perfacul = main6.span.text
        except Exception as e:
            perfacul = 'None'
        subfacul.append(perfacul)

    #Nextpage link
    for main7 in soup4.find_all('li', class_='next'):
        try:
            nextpage = main7.a['href']
        except Exception as e:  
            nextpage = 'None'
        #nextpag.append(nextpage)
      

    b ={'mainfaculty':faculty, 'pernames':pnames,'subfaculty':subfacul}
    cov_facul = pd.DataFrame.from_dict(b, orient='index')
    cov_facult = cov_facul.transpose()
    cov_facult['faculty'] =cov_facult['mainfaculty'].iloc[0]
    cov_facult.to_csv('cov_faculty_art.csv', mode='a', header=False)
    
    nextpagelink = ("https://pureportal.coventry.ac.uk" + str(nextpage))       
    print(nextpagelink)
    sleep(randint(1,5))
    j += 1
   

https://pureportal.coventry.ac.uk/en/organisations/faculty-of-arts-humanities/persons/?page=2
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-arts-humanities/persons/?page=3
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-arts-humanities/persons/?page=4
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-arts-humanities/persons/?page=5
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-arts-humanities/persons/?page=5


In [11]:
cov_facult.head()

,mainfaculty,pernames,subfaculty,faculty
0,Faculty of Arts & Humanities,Jo Sperryn-Jones,School of Art and Design,Faculty of Arts & Humanities
1,None,Esme Spurling,School of Media and Performing Arts,Faculty of Arts & Humanities
2,None,Kathryn Stamp,Faculty Research Centre for Dance Research (CD...,Faculty of Arts & Humanities
3,None,Kaori Stansall,School of Humanities,Faculty of Arts & Humanities
4,None,Jo Gane Steggall,School of Media and Performing Arts,Faculty of Arts & Humanities


### Faculty of Business and Law

In [12]:
faculty= []
subfacul= []
pnames = []
nextpag = []

page4 = requests.get("https://pureportal.coventry.ac.uk/en/organisations/faculty-of-business-law/persons/")
soup4 = BeautifulSoup(page4.text, 'html.parser')


for main4 in soup4.find_all('div', class_='organisation-details'):
    try:
        facul = main4.h1.text
    except Exception as e:
        facul = 'None'
    faculty.append(facul)    
    
for main5 in soup4.find_all('h3', class_='title'):
    try:
        pers = main5.text
    except Exception as e:
        pers = 'None'
    pnames.append(pers)    
        
for main6 in soup4.find_all('ul',class_='relations organisations'):
    try:
        perfacul = main6.span.text
    except Exception as e:
        perfacul = 'None'
    subfacul.append(perfacul)

#Nextpage link
for main7 in soup4.find_all('li', class_='next'):
    try:
        nextpage = main7.a['href']
    except Exception as e:  
        nextpage = 'None'
    #nextpag.append(nextpage)
    
    nextpagelink = ("https://pureportal.coventry.ac.uk" + str(nextpage))       
    print(nextpagelink)
    
    b ={'mainfaculty':faculty, 'pernames':pnames,'subfaculty':subfacul}
    cov_facul = pd.DataFrame.from_dict(b, orient='index')
    cov_facult = cov_facul.transpose()
    cov_facult['faculty'] =cov_facult['mainfaculty'].iloc[0]
    cov_facult.to_csv('cov_faculty_bus.csv')


https://pureportal.coventry.ac.uk/en/organisations/faculty-of-business-law/persons/?page=1


In [13]:
j = 1
while j < 10:
    faculty= []
    subfacul= []
    pnames = []
    nextpag = []

    page4 = requests.get(nextpagelink)
    soup4 = BeautifulSoup(page4.text, 'html.parser')


    for main4 in soup4.find_all('div', class_='organisation-details'):
        try:
            facul = main4.h1.text
        except Exception as e:
            facul = 'None'
        faculty.append(facul)    

    for main5 in soup4.find_all('h3', class_='title'):
        try:
            pers = main5.text
        except Exception as e:
            pers = 'None'
        pnames.append(pers)    

    for main6 in soup4.find_all('ul',class_='relations organisations'):
        try:
            perfacul = main6.span.text
        except Exception as e:
            perfacul = 'None'
        subfacul.append(perfacul)

    #Nextpage link
    for main7 in soup4.find_all('li', class_='next'):
        try:
            nextpage = main7.a['href']
        except Exception as e:  
            nextpage = 'None'
        #nextpag.append(nextpage)        

    b ={'mainfaculty':faculty, 'pernames':pnames,'subfaculty':subfacul}
    cov_facul = pd.DataFrame.from_dict(b, orient='index')
    cov_facult = cov_facul.transpose()
    cov_facult['faculty'] =cov_facult['mainfaculty'].iloc[0]
    cov_facult.to_csv('cov_faculty_bus.csv', mode='a', header=False)


    nextpagelink = ("https://pureportal.coventry.ac.uk" + str(nextpage))       
    print(nextpagelink)
    sleep(randint(1,5))
    j += 1


https://pureportal.coventry.ac.uk/en/organisations/faculty-of-business-law/persons/?page=2
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-business-law/persons/?page=3
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-business-law/persons/?page=4
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-business-law/persons/?page=5
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-business-law/persons/?page=6
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-business-law/persons/?page=7
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-business-law/persons/?page=8
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-business-law/persons/?page=9
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-business-law/persons/?page=9


### Faculty of Engineering, Environment and Computing 

In [14]:
faculty= []
subfacul= []
pnames = []
nextpag = []

page4 = requests.get("https://pureportal.coventry.ac.uk/en/organisations/faculty-of-engineering-environment-computing/persons/")
soup4 = BeautifulSoup(page4.text, 'html.parser')


for main4 in soup4.find_all('div', class_='organisation-details'):
    try:
        facul = main4.h1.text
    except Exception as e:
        facul = 'None'
    faculty.append(facul)    
    
for main5 in soup4.find_all('h3', class_='title'):
    try:
        pers = main5.text
    except Exception as e:
        pers = 'None'
    pnames.append(pers)    
        
for main6 in soup4.find_all('ul',class_='relations organisations'):
    try:
        perfacul = main6.span.text
    except Exception as e:
        perfacul = 'None'
    subfacul.append(perfacul)

#Nextpage link
for main7 in soup4.find_all('li', class_='next'):
    try:
        nextpage = main7.a['href']
    except Exception as e:  
        nextpage = 'None'
    #nextpag.append(nextpage)
    
    nextpagelink = ("https://pureportal.coventry.ac.uk" + str(nextpage))       
    print(nextpagelink)
    
    b ={'mainfaculty':faculty, 'pernames':pnames,'subfaculty':subfacul}
    cov_facul = pd.DataFrame.from_dict(b, orient='index')
    cov_facult = cov_facul.transpose()
    cov_facult['faculty'] =cov_facult['mainfaculty'].iloc[0]
    cov_facult.to_csv('cov_faculty_eng.csv')


https://pureportal.coventry.ac.uk/en/organisations/faculty-of-engineering-environment-computing/persons/?page=1


In [15]:
h = 1
while h < 7:
    faculty= []
    subfacul= []
    pnames = []
    nextpag = []

    page4 = requests.get(nextpagelink)
    soup4 = BeautifulSoup(page4.text, 'html.parser')


    for main4 in soup4.find_all('div', class_='organisation-details'):
        try:
            facul = main4.h1.text
        except Exception as e:
            facul = 'None'
        faculty.append(facul)    

    for main5 in soup4.find_all('h3', class_='title'):
        try:
            pers = main5.text
        except Exception as e:
            pers = 'None'
        pnames.append(pers)    

    for main6 in soup4.find_all('ul',class_='relations organisations'):
        try:
            perfacul = main6.span.text
        except Exception as e:
            perfacul = 'None'
        subfacul.append(perfacul)

    #Nextpage link
    for main7 in soup4.find_all('li', class_='next'):
        try:
            nextpage = main7.a['href']
        except Exception as e:  
            nextpage = 'None'
        #nextpag.append(nextpage)        

    b ={'mainfaculty':faculty, 'pernames':pnames,'subfaculty':subfacul}
    cov_facul = pd.DataFrame.from_dict(b, orient='index')
    cov_facult = cov_facul.transpose()
    cov_facult['faculty'] =cov_facult['mainfaculty'].iloc[0]
    cov_facult.to_csv('cov_faculty_eng.csv', mode='a', header=False)


    nextpagelink = ("https://pureportal.coventry.ac.uk" + str(nextpage))       
    print(nextpagelink)
    sleep(randint(1,5))
    h += 1


https://pureportal.coventry.ac.uk/en/organisations/faculty-of-engineering-environment-computing/persons/?page=2
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-engineering-environment-computing/persons/?page=3
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-engineering-environment-computing/persons/?page=4
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-engineering-environment-computing/persons/?page=5
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-engineering-environment-computing/persons/?page=6
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-engineering-environment-computing/persons/?page=6


### Faculty of Health and Life Sciences

In [16]:
faculty= []
subfacul= []
pnames = []
nextpag = []

page4 = requests.get("https://pureportal.coventry.ac.uk/en/organisations/faculty-of-health-life-sciences/persons/")
soup4 = BeautifulSoup(page4.text, 'html.parser')


for main4 in soup4.find_all('div', class_='organisation-details'):
    try:
        facul = main4.h1.text
    except Exception as e:
        facul = 'None'
    faculty.append(facul)    
    
for main5 in soup4.find_all('h3', class_='title'):
    try:
        pers = main5.text
    except Exception as e:
        pers = 'None'
    pnames.append(pers)    
        
for main6 in soup4.find_all('ul',class_='relations organisations'):
    try:
        perfacul = main6.span.text
    except Exception as e:
        perfacul = 'None'
    subfacul.append(perfacul)

#Nextpage link
for main7 in soup4.find_all('li', class_='next'):
    try:
        nextpage = main7.a['href']
    except Exception as e:  
        nextpage = 'None'
    #nextpag.append(nextpage)
    
    nextpagelink = ("https://pureportal.coventry.ac.uk" + str(nextpage))       
    print(nextpagelink)
    
    b ={'mainfaculty':faculty, 'pernames':pnames,'subfaculty':subfacul}
    cov_facul = pd.DataFrame.from_dict(b, orient='index')
    cov_facult = cov_facul.transpose()
    cov_facult['faculty'] =cov_facult['mainfaculty'].iloc[0]
    cov_facult.to_csv('cov_faculty_hea.csv')


https://pureportal.coventry.ac.uk/en/organisations/faculty-of-health-life-sciences/persons/?page=1


In [17]:
k = 1
while k < 10:
    faculty= []
    subfacul= []
    pnames = []
    nextpag = []

    page4 = requests.get(nextpagelink)
    soup4 = BeautifulSoup(page4.text, 'html.parser')


    for main4 in soup4.find_all('div', class_='organisation-details'):
        try:
            facul = main4.h1.text
        except Exception as e:
            facul = 'None'
        faculty.append(facul)    

    for main5 in soup4.find_all('h3', class_='title'):
        try:
            pers = main5.text
        except Exception as e:
            pers = 'None'
        pnames.append(pers)    

    for main6 in soup4.find_all('ul',class_='relations organisations'):
        try:
            perfacul = main6.span.text
        except Exception as e:
            perfacul = 'None'
        subfacul.append(perfacul)

    #Nextpage link
    for main7 in soup4.find_all('li', class_='next'):
        try:
            nextpage = main7.a['href']
        except Exception as e:  
            nextpage = 'None'
        #nextpag.append(nextpage)        

    b ={'mainfaculty':faculty, 'pernames':pnames,'subfaculty':subfacul}
    cov_facul = pd.DataFrame.from_dict(b, orient='index')
    cov_facult = cov_facul.transpose()
    cov_facult['faculty'] =cov_facult['mainfaculty'].iloc[0]
    cov_facult.to_csv('cov_faculty_hea.csv', mode='a', header=False)


    nextpagelink = ("https://pureportal.coventry.ac.uk" + str(nextpage))       
    print(nextpagelink)
    sleep(randint(1,5))
    k += 1


https://pureportal.coventry.ac.uk/en/organisations/faculty-of-health-life-sciences/persons/?page=2
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-health-life-sciences/persons/?page=3
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-health-life-sciences/persons/?page=4
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-health-life-sciences/persons/?page=5
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-health-life-sciences/persons/?page=6
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-health-life-sciences/persons/?page=7
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-health-life-sciences/persons/?page=8
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-health-life-sciences/persons/?page=9
https://pureportal.coventry.ac.uk/en/organisations/faculty-of-health-life-sciences/persons/?page=9


### Coventry research personnel research Interest

In [18]:
stflinks = []
names = []

page5 = requests.get("https://pureportal.coventry.ac.uk/en/persons/")
soup5 = BeautifulSoup(page5.text, 'html.parser')


for main5 in soup5.find_all('h3', class_='title'):
    try:
        slink = main5.a['href']
    except Exception as e:
        slink = 'None' 
    stflinks.append(slink)
    
    try:
        stfname=main5.text
    except Exception as e:
        stfname = 'None' 
    names.append(stfname)
    
for main6 in soup5.find_all('li', class_='next'):
    try:
        nextpg = main6.a['href']
    except Exception as e:
        nextpg = 'None'
            
headers = []
interest= []
required = []
for stfpage in stflinks:    
    page6 = requests.get(stfpage)
    soup6 = BeautifulSoup(page6.text, 'html.parser')
    try:
        header = soup6.find_all('h3', class_='subheader')[1].text                     
    except Exception as e:
        header = 'None'     
    try:
        container = soup6.find_all('div',class_='textblock')[1].text 
    except Exception as e:
        container = 'None'     
    requir = header + ': ' + container    
    required.append(requir)
    
          
reasearch = pd.DataFrame({
    'names_2': names,
    'link_CU' : stflinks,
    'research_interest': required
})
reasearch.to_csv('cov_reasearch.csv')

links = ("https://pureportal.coventry.ac.uk" + str(nextpg))
print(links)     
   

https://pureportal.coventry.ac.uk/en/persons/?format=&page=1


In [19]:
k = 1
while k < 41:
    stflinks = []
    names = []

    page5 = requests.get(links)
    soup5 = BeautifulSoup(page5.text, 'html.parser')


    for main5 in soup5.find_all('h3', class_='title'):
        try:
            slink = main5.a['href']
        except Exception as e:
            slink = 'None' 
        stflinks.append(slink)

        try:
            stfname=main5.text
        except Exception as e:
            stfname = 'None' 
        names.append(stfname)

    for main6 in soup5.find_all('li', class_='next'):
        try:
            nextpg = main6.a['href']
        except Exception as e:
            nextpg = 'None'

    headers = []
    interest= []
    required = []
    for stfpage in stflinks:    
        page6 = requests.get(stfpage)
        soup6 = BeautifulSoup(page6.text, 'html.parser')
        try:
            header = soup6.find_all('h3', class_='subheader')[1].text                     
        except Exception as e:
            header = 'None'     
        try:
            container = soup6.find_all('div',class_='textblock')[1].text 
        except Exception as e:
            container = 'None'     
        requir = header + ': ' + container    
        required.append(requir)


    reasearch = pd.DataFrame({
        'names_2': names,
        'link_CU' : stflinks,
        'research_interest': required
    })
    reasearch.to_csv('cov_reasearch.csv', mode='a', header=False)

    links = ("https://pureportal.coventry.ac.uk" + str(nextpg))
    #print(links)
    sleep(randint(1,5))
    k += 1   
    

### Combine the Faculty csv's

In [20]:
import pandas as pd
import csv

Finded_URL = ['cov_faculty_art.csv','cov_faculty_bus.csv','cov_faculty_eng.csv','cov_faculty_hea.csv' ]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f,header=0) for f in Finded_URL])
combined_csv.head()
combined_csv.to_csv( "Finded_URL_All.csv", quotechar='"',
                    quoting=csv.QUOTE_ALL, index=False, encoding='utf-8')